In [11]:
%load_ext autoreload
%autoreload 2

import importlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import mlflow

## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

    What's the version that you have?

`(mlflow) michal@pop-os:~/Projects/mlops-zoomcamp$ mlflow --version`

`mlflow, version 1.26.0`


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset.

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7


In [18]:
# use brace expansion to download all January - March files
!wget -P data 'https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-0'{1..3}'.parquet'

--2022-05-30 15:44:38--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.225.200
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.225.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-01.parquet’

green_tripdata_2021 100%[===================>]   1.27M   192KB/s    in 9.6s    

2022-05-30 15:44:49 (135 KB/s) - ‘data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-05-30 15:44:49--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Reusing existing connection to s3.amazonaws.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-02.parquet’

green_tripdata_2021 100%[===================>]   1.09M   265KB/s    in 4.3s    

2022-05-30 15:44:54 (262 KB/s) - ‘data/green_tripdata_2021-

### Preprocessing

In [23]:
!python ./homework/preprocess_data.py --raw_data_path ./data/ --dest_path ./output

In [27]:
# How many files are in the output direcotry ?
!dir ./output | wc -w

4


## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

* 19
* 17
* 10
* 20

In [29]:
!python ./homework/train.py --data_path ./output

tracking URI: 'http://127.0.0.1:5000'
2022/05/30 16:42:42 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment-hw2' does not exist. Creating a new experiment.
2022/05/30 16:42:42 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/30 16:42:42 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/30 16:42:54 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/michal/.virtualenvs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [41]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI="http://127.0.0.1:5000"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment-hw2', tags={}>]

In [36]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    #filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, params: {run.data._params, len(run.data._params)}")


run id: 8a5031d014564f94a4a77fd78668943a, params: ({'bootstrap': 'True', 'ccp_alpha': '0.0', 'criterion': 'squared_error', 'max_depth': '10', 'max_features': '1.0', 'max_leaf_nodes': 'None', 'max_samples': 'None', 'min_impurity_decrease': '0.0', 'min_samples_leaf': '1', 'min_samples_split': '2', 'min_weight_fraction_leaf': '0.0', 'n_estimators': '100', 'n_jobs': 'None', 'oob_score': 'False', 'random_state': '0', 'verbose': '0', 'warm_start': 'False'}, 17)


Answer: 17

## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`



`mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 0.0.0.0 --port 5000`
Answer: default-artifact-root

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise.

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

* 6.128
* 6.628
* 7.128
* 7.628


In [44]:
# mlflow.exceptions.MlflowException: Cannot set a deleted experiment 'random-forest-hyperopt' as the active experiment. You can restore the experiment, or permanently delete the  experiment to create a new one.

In [55]:
!python homework/hpo.py

100%|█████████| 50/50 [03:54<00:00,  4.68s/trial, best loss: 6.6284257482044735]


List runs for new experiment

In [56]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment-hw2', tags={}>,
 <Experiment: artifact_location='./mlruns/3', experiment_id='3', lifecycle_stage='active', name='random-forest-hyperopt-hw2', tags={}>]

In [57]:
runs = client.search_runs(
    experiment_ids='3',
    #filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [58]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 3a6adf5eabe4413c899335cf8c6fbc6c, rmse: 6.6284
run id: e9c7a04fe6674e5b95f52cb79e7c4212, rmse: 6.6297
run id: 6d6050f27a2049599002560598c10ab1, rmse: 6.6299
run id: f68a76149781499b936114515ec137bb, rmse: 6.6299
run id: 8a1b4867bc7c4812a3a748b423b224c5, rmse: 6.6299


### What's the best validation RMSE that you got?
 6.6284

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.
Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 6.1
* 6.55
* 7.93
* 15.1

In [59]:
!python homework/register_model.py

2022/05/30 18:43:46 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models-hw2' does not exist. Creating a new experiment.
2022/05/30 18:43:46 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/30 18:43:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/michal/.virtualenvs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'nyc-taxi-rf-regressor'.
2022/05/30 18:44:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-rf-regressor, version 1
Created version '1' of model 'nyc-taxi-rf-regressor'.


In [78]:
registered_models = client.list_registered_models()
registered_models[0].latest_versions[0]

<ModelVersion: creation_timestamp=1653929076544, current_stage='None', description='', last_updated_timestamp=1653929076544, name='nyc-taxi-rf-regressor', run_id='19a73b6f3dd443bbaa72987a1c9a670e', run_link='', source='./mlruns/4/19a73b6f3dd443bbaa72987a1c9a670e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [80]:
run = client.search_runs(
    experiment_ids='4',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_rmse ASC"]
)[0]
print(f"run id: {run.info.run_id}, test_rmse: {run.data.metrics['test_rmse']:.4f}")

run id: 19a73b6f3dd443bbaa72987a1c9a670e, test_rmse: 6.5479
